# Initialization

*Please read carefully*. In order to run the notebook successfully, you need to follow a few steps:
1. Download the raw data you'd like to load to the appropriate location on your computer.
2. Run the code below to import the necessary packages.
3. Determine whether you would like to load data to your local instance of the app (the one you run via `flask run` in your terminal at http://127.0.0.1:5000) or to the live online database (https://sspi.world).

## 1. Set the Filepath

In order to load data, you first have to tell your computer where it is.  I keep my data in the `bulk_data` folder, which is a sibling of this file (it is in the same folder).  You will notice, however, that you can't see any of my data files.  This is not an accident.  GitHub does not like to store large files, so I have `.gitignore`d the contents of the `bulk_data` folder. What this means is that git will not keep track of the contents of this folder, and will not upload or download it.  This means that it will be your responsibility to download the data files from the Google Drive and to organize the folder appropriately.  I recommend following the SSPI hierarchy, with three folders for the three pillars, nested folders for the categories, and further nested folders for the indicators.  Inside of the indicator folders, you can keep the raw(s) file for the data.

To import data from this folder, you'll use a relative import statement. It will start at the current directory (the one this file is in) and then move down the folder hierarchy.  The import statement below looks in the current folder with `./`, then looks to the subfolder `bulk_data`, then down to `SUS` for the Sustainability Pillar, then `LND` for the Land Category, and finally in the `NITROG` folder for the Sustainable Nitrogen Management indicator.  Into that folder I have downloaded the file from the source below, which itself contains a folder structure.  It have intentionally not cleaned up any of the files or folder structure from the download to make it as replicable as possible.

In [31]:
datapath = r"./raw_bulk_data/SUS/LND/NITROG/2022-epi-indicators-time-series-csv/2022-epi-indicators-time-series/SNM_ind.csv"

## 2. Import Packages

The code below imports the standard python data manipulation packages.  If you need to add a package for something, you can add it here.

In [32]:
import pandas as pd
import numpy as np
import re

## 3. Connect to the Database

This part is only important once you've run your analysis and would like to connect up to the database. The code below imports the `SSPIDatabaseConnector`, which is a Python class that allows us to post data directly to our databases. You can use this class to post data to both your development `sspi_bulk_data` (the one that lives on your machine that interfaces with the app you get when you run `flask run`) and the production `sspi_bulk_data` database, which lives on the server which hosts the live app at https://sspi.world. In order to access the production database, you'll need to ask me for your API token.  To access your development database, you can find the API token associated with your login credentials (the ones you use to login into the backend of the site) by visiting http://127.0.0.1:5000/auth/query while logged in, which will print out your API token.  

**It is vitally important that you _NEVER_ commit your API Tokens! They should only ever be stored in your local `.env` file, which is `.gitignore`d in this repository.** Never paste an API Token in any file unless you are absolutely sure it's the right place. Just don't do it!

In [33]:
from sys import path
path.append('../')

from database_connector.SSPIDatabaseConnector import SSPIDatabaseConnector
database = SSPIDatabaseConnector()

# I. Data Source

The 2018 SSPI originally used the Nitrogen Management Index sourced through the Environmental Performance Index (EPI) from their website, https://epi.yale.edu/.  The original values sourced for the SSPI were copied and pasted by hand from the EPI website.  The EPI website has since been updated, and the page from which the data were copied no longer exists.  The 2022 version of the page is available at https://epi.yale.edu/epi-results/2022/component/snm.

Fortunately, https://epi.yale.edu/downloads has a blurb which contains a link to https://sedac.ciesin.columbia.edu/data/set/epi-environmental-performance-index-2022/data-download for bulk data downloads.  The most recent bulk download is for 2022, and covers the years 1950 - 2022.

There are also previous years available for download.  It may be an interesting project to compare the EPI datasets over the years to see how they revise data and whether we think their methodology produces reliable results.  For now, we will focus on the 2022 dataset, which I have downloaded locally and uploaded to the Google Drive at https://drive.google.com/drive/folders/102Lg4NREiRhH_pzLabiVbkHvxRiaepDq?usp=sharing

In [34]:
SNM_raw_indicators = pd.read_csv(datapath)
SNM_raw_indicators

,code,iso,country,SNM.ind.1995,SNM.ind.1996,SNM.ind.1997,SNM.ind.1998,SNM.ind.1999,SNM.ind.2000,SNM.ind.2001,...,SNM.ind.2013,SNM.ind.2014,SNM.ind.2015,SNM.ind.2016,SNM.ind.2017,SNM.ind.2018,SNM.ind.2019,SNM.ind.2020,SNM.ind.2021,SNM.ind.2022
0,4,AFG,Afghanistan,44.015857,44.940128,46.931886,47.409220,45.026760,46.935768,46.024914,...,49.654873,52.474220,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194
1,8,ALB,Albania,32.740191,32.877496,35.064258,32.049513,32.014128,33.975968,32.473496,...,40.553903,40.455314,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271
2,12,DZA,Algeria,39.122933,33.470399,28.022457,38.560891,37.182622,27.406628,40.748135,...,49.653891,42.949184,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487
3,20,AND,Andorra,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
4,24,AGO,Angola,24.672298,24.974387,25.923148,27.375270,28.690846,30.976644,32.359858,...,34.652635,30.103431,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,876,WLF,Wallis and Futuna Islands,6.511117,6.690118,6.809275,6.930870,7.049151,7.223545,7.367040,...,9.631419,9.441148,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377
216,732,ESH,Western Sahara,22.890672,21.792781,23.324610,21.604391,23.701670,18.721917,8.913881,...,15.001656,15.020989,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210
217,887,YEM,Yemen,40.789860,40.570173,39.305332,42.575937,42.763620,42.499581,42.102089,...,40.567349,39.558267,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760
218,894,ZMB,Zambia,26.058138,41.955960,31.324801,34.640796,41.127625,44.265747,37.610371,...,36.999181,40.853596,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438


In [35]:
SNM_raw_indicators = SNM_raw_indicators.drop(columns=['code', 'country'])
SNM_raw_indicators

,iso,SNM.ind.1995,SNM.ind.1996,SNM.ind.1997,SNM.ind.1998,SNM.ind.1999,SNM.ind.2000,SNM.ind.2001,SNM.ind.2002,SNM.ind.2003,...,SNM.ind.2013,SNM.ind.2014,SNM.ind.2015,SNM.ind.2016,SNM.ind.2017,SNM.ind.2018,SNM.ind.2019,SNM.ind.2020,SNM.ind.2021,SNM.ind.2022
0,AFG,44.015857,44.940128,46.931886,47.409220,45.026760,46.935768,46.024914,49.757632,49.370340,...,49.654873,52.474220,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194
1,ALB,32.740191,32.877496,35.064258,32.049513,32.014128,33.975968,32.473496,28.272164,26.632317,...,40.553903,40.455314,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271
2,DZA,39.122933,33.470399,28.022457,38.560891,37.182622,27.406628,40.748135,40.608845,32.869971,...,49.653891,42.949184,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487
3,AND,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
4,AGO,24.672298,24.974387,25.923148,27.375270,28.690846,30.976644,32.359858,33.645186,33.379226,...,34.652635,30.103431,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,WLF,6.511117,6.690118,6.809275,6.930870,7.049151,7.223545,7.367040,7.916157,7.634675,...,9.631419,9.441148,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377
216,ESH,22.890672,21.792781,23.324610,21.604391,23.701670,18.721917,8.913881,18.044591,18.741216,...,15.001656,15.020989,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210
217,YEM,40.789860,40.570173,39.305332,42.575937,42.763620,42.499581,42.102089,41.175425,39.726230,...,40.567349,39.558267,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760
218,ZMB,26.058138,41.955960,31.324801,34.640796,41.127625,44.265747,37.610371,25.658239,34.430729,...,36.999181,40.853596,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438


In [49]:
SNM_raw_indicators = SNM_raw_indicators.rename(columns={'iso': 'CountryCode'})
SNM_raw_indicators

,CountryCode,SNM.ind.1995,SNM.ind.1996,SNM.ind.1997,SNM.ind.1998,SNM.ind.1999,SNM.ind.2000,SNM.ind.2001,SNM.ind.2002,SNM.ind.2003,...,SNM.ind.2013,SNM.ind.2014,SNM.ind.2015,SNM.ind.2016,SNM.ind.2017,SNM.ind.2018,SNM.ind.2019,SNM.ind.2020,SNM.ind.2021,SNM.ind.2022
0,AFG,44.015857,44.940128,46.931886,47.409220,45.026760,46.935768,46.024914,49.757632,49.370340,...,49.654873,52.474220,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194
1,ALB,32.740191,32.877496,35.064258,32.049513,32.014128,33.975968,32.473496,28.272164,26.632317,...,40.553903,40.455314,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271
2,DZA,39.122933,33.470399,28.022457,38.560891,37.182622,27.406628,40.748135,40.608845,32.869971,...,49.653891,42.949184,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487
3,AND,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
4,AGO,24.672298,24.974387,25.923148,27.375270,28.690846,30.976644,32.359858,33.645186,33.379226,...,34.652635,30.103431,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,WLF,6.511117,6.690118,6.809275,6.930870,7.049151,7.223545,7.367040,7.916157,7.634675,...,9.631419,9.441148,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377
216,ESH,22.890672,21.792781,23.324610,21.604391,23.701670,18.721917,8.913881,18.044591,18.741216,...,15.001656,15.020989,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210
217,YEM,40.789860,40.570173,39.305332,42.575937,42.763620,42.499581,42.102089,41.175425,39.726230,...,40.567349,39.558267,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760
218,ZMB,26.058138,41.955960,31.324801,34.640796,41.127625,44.265747,37.610371,25.658239,34.430729,...,36.999181,40.853596,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438


In [50]:
SNM_long_indicators = SNM_raw_indicators.melt(id_vars=['CountryCode'], var_name='YearString', value_name='Raw')
SNM_long_indicators

,CountryCode,YearString,Raw
0,AFG,SNM.ind.1995,44.015857
1,ALB,SNM.ind.1995,32.740191
2,DZA,SNM.ind.1995,39.122933
3,AND,SNM.ind.1995,-8888.000000
4,AGO,SNM.ind.1995,24.672298
...,...,...,...
6155,WLF,SNM.ind.2022,9.452377
6156,ESH,SNM.ind.2022,14.788210
6157,YEM,SNM.ind.2022,34.527760
6158,ZMB,SNM.ind.2022,36.637438


In [51]:
SNM_long_indicators["Year"] = [re.search(r"\d{4}", s).group(0) for s in SNM_long_indicators["YearString"]]
SNM_long_indicators


,CountryCode,YearString,Raw,Year
0,AFG,SNM.ind.1995,44.015857,1995
1,ALB,SNM.ind.1995,32.740191,1995
2,DZA,SNM.ind.1995,39.122933,1995
3,AND,SNM.ind.1995,-8888.000000,1995
4,AGO,SNM.ind.1995,24.672298,1995
...,...,...,...,...
6155,WLF,SNM.ind.2022,9.452377,2022
6156,ESH,SNM.ind.2022,14.788210,2022
6157,YEM,SNM.ind.2022,34.527760,2022
6158,ZMB,SNM.ind.2022,36.637438,2022


In [52]:
SNM_long_indicators.drop(columns=['YearString'], inplace=True)
SNM_long_indicators

,CountryCode,Raw,Year
0,AFG,44.015857,1995
1,ALB,32.740191,1995
2,DZA,39.122933,1995
3,AND,-8888.000000,1995
4,AGO,24.672298,1995
...,...,...,...
6155,WLF,9.452377,2022
6156,ESH,14.788210,2022
6157,YEM,34.527760,2022
6158,ZMB,36.637438,2022


In [54]:
SNM_long_indicators.drop(SNM_long_indicators[SNM_long_indicators['Raw'] < 0].index, inplace=True)
SNM_long_indicators

,CountryCode,Raw,Year
0,AFG,44.015857,1995
1,ALB,32.740191,1995
2,DZA,39.122933,1995
4,AGO,24.672298,1995
6,ATG,2.418811,1995
...,...,...,...
6155,WLF,9.452377,2022
6156,ESH,14.788210,2022
6157,YEM,34.527760,2022
6158,ZMB,36.637438,2022


In [55]:
SNM_long_indicators.assign(IndicatorCode="NITROG")

,CountryCode,Raw,Year,IndicatorCode
0,AFG,44.015857,1995,NITROG
1,ALB,32.740191,1995,NITROG
2,DZA,39.122933,1995,NITROG
4,AGO,24.672298,1995,NITROG
6,ATG,2.418811,1995,NITROG
...,...,...,...,...
6155,WLF,9.452377,2022,NITROG
6156,ESH,14.788210,2022,NITROG
6157,YEM,34.527760,2022,NITROG
6158,ZMB,36.637438,2022,NITROG
